In [18]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [21]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

In [20]:
series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50)), 
                         '2': pd.Series(np.arange(start=50, stop=100))})

In [22]:
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=[3, 5])

forecaster = ForecasterAutoregMultiSeries(
    LinearRegression(), lags=3, window_features=rolling
)
forecaster.fit(series=series_2)

c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:431: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


In [28]:
steps = 1
inputs_predict = forecaster._create_predict_inputs(steps=steps)

In [30]:
inputs_predict[0]

,1,2
45,45,95
46,46,96
47,47,97
48,48,98
49,49,99


In [31]:
levels = inputs_predict[2]
last_window = inputs_predict[0]
exog_values_dict = inputs_predict[1]

forecaster._recursive_predict(
    steps=steps, levels=levels, last_window=last_window, exog_values_dict=exog_values_dict
)

[[45. 95.]
 [46. 96.]
 [47. 97.]
 [48. 98.]
 [49. 99.]]
[[ 49.  48.  47.  73. 720.   0.]
 [ 99.  98.  97.  73. 720.   1.]]


array([[134.48275862, 139.65517241]])

In [69]:
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=[3, 5])
rolling.transform_2d(_)

array([[ 48., 235.],
       [ 98., 485.]])

In [70]:
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=[3, 5])
rolling.transform_2d(_.flatten())

array([[ 82., 391.]])

In [73]:
np.concatenate([rolling.transform_2d(_.flatten()).ravel(), 
                rolling.transform_2d(_.flatten()).ravel()])

array([ 82., 391.,  82., 391.])

In [49]:
_ = inputs_predict[0].to_numpy()[-5:]
_

array([[45, 95],
       [46, 96],
       [47, 97],
       [48, 98],
       [49, 99]])

In [55]:
_2 = _.astype(float)
_2[1, 1] = np.nan
_2

array([[45., 95.],
       [46., nan],
       [47., 97.],
       [48., 98.],
       [49., 99.]])

In [57]:
_2[~np.isnan(_2).any(axis=1)]

array([[45., 95.],
       [47., 97.],
       [48., 98.],
       [49., 99.]])

In [68]:
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=[3, 5])
rolling.transform_2d(_)

array([[ 48., 235.],
       [ 98., 485.]])

In [65]:
np.arange(3).reshape(1, -1)

array([[0, 1, 2]])

In [66]:
np.array([[0, 1, 2]]).reshape(1, -1)

array([[0, 1, 2]])

In [83]:
a = np.arange(1000)

In [84]:
%%timeit

a.reshape(-1, 1)

232 ns ± 8.78 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [85]:
a = np.arange(1000)

In [86]:
%%timeit

a[:, np.newaxis]

185 ns ± 9.38 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [89]:
y = np.array([1, 4, 8, 10, 13, 22, 40, 46], dtype=float)
multiple_series = np.concatenate([y, y], axis=0).reshape(2, -1).T
multiple_series

array([[ 1.,  1.],
       [ 4.,  4.],
       [ 8.,  8.],
       [10., 10.],
       [13., 13.],
       [22., 22.],
       [40., 40.],
       [46., 46.]])

In [100]:
y = np.array([1, 4, 8, 10, 13, 22, 40, 46], dtype=float)
next_window_diff = np.array([[9., 9.], [15., 15.], [1., 1.]], dtype=float)
next_window = np.array([55, 70, 71], dtype=float)
next_window_series = np.concatenate([next_window, next_window], axis=0).reshape(2, -1).T
next_window_series

array([[55., 55.],
       [70., 70.],
       [71., 71.]])

In [99]:
transformer = TimeSeriesDifferentiator(order=1)
transformer.fit_transform(y)
results = transformer.inverse_transform_next_window(next_window_diff)
results

array([[55., 55.],
       [70., 70.],
       [71., 71.]])

In [101]:
np.arange(10).shape

(10,)